In [179]:
import pickle
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from joblib import dump

In [180]:
def resize_imgs(imgs, new_shape):
    imgs_resized = []
    
    for img in imgs:
            imgs_resized.append(cv.resize(img, new_shape))
    
    return np.array(imgs_resized)

In [181]:
def print_scores_cv(clf, x, y, n_fold):
    scores = cross_val_score(clf, x_ravel, y, cv=n_fold, n_jobs=4)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    return scores

# Importing files

In [182]:
with open("imgs_me", "rb") as img_my:
    imgs_myface = pickle.load(img_my)
    
with open("295_faces", "rb") as img_no:
    imgs_nomyface = pickle.load(img_no)

# Creating Dataset

In [185]:
x = resize_imgs(imgs_myface + imgs_nomyface, (110, 110))
y = np.hstack((np.ones(len(imgs_myface)), np.zeros(len(imgs_nomyface))))

x_ravel = x.reshape(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
x_ravel_norm = normalize(x_ravel)

# Evaluating Classifiers w/ 5-fold Cross Validation
## AdaBoost (Boosted Decision Trees)

In [ ]:
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=10000)
scores_ada = print_scores_cv(clf, x_ravel_norm, y, 5)

## SVM (RBF Kernel)

In [ ]:
svm = SVC(gamma='scale')
scores_svm = print_scores_cv(svm, x_ravel_norm, y, 5)

## Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=150, n_jobs=4)
scores_rf = print_scores_cv(rf, x_ravel_norm, y, 5)

## Naive Bayes

In [ ]:
nb = GaussianNB()
scores_nb = print_scores_cv(nb, x_ravel_norm, y, 5)

## Neural Network (MLP)

In [161]:
mlp = MLPClassifier(hidden_layer_sizes=200)
x_train, x_test, y_train, y_test = train_test_split(x_ravel_norm, y, test_size=0.1)

mlp.fit(x_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=200, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [162]:
mlp.score(x_test, y_test)

1.0

# K-Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier()
scores_knn = print_scores_cv(knn, x, y, 5)

In [170]:
clf.fit(x_ravel, y)
dump(clf, "adaboost_face.ada")

['adaboost_face.ada']

In [174]:
clf.predict(x_ravel[0].reshape(1, x_ravel[0].shape[0]))

array([1.])

In [175]:
rf.fit(x_ravel, y)
dump(rf, "randomforest_face.rf")

['randomforest_face.rf']

In [177]:
nb.fit(x_ravel, y)
dump(nb, "naivebayes_face.nb")

['naivebayes_face.nb']

In [178]:
mlp.fit(x_ravel, y)
dump(mlp, "mlp_face.mlp")

['mlp_face.mlp']

In [ ]:
t